# Image Similarity Metrics Analysis for NIfTI Images

1. **Load NIfTI Files:**  
The code initially specifies paths to the fixed (or template) image and three other images (moving, 25% down-sampled registered, and 50% down-sampled registered). 
2. **Calculate MI and ZNCC:**  
calculate_metrics: This function computes two similarity metrics - Mutual Information (MI) and Zero-Normalized Cross-Correlation (ZNCC) - for two given images.  
MI is a measure of the amount of information one can obtain about one image by observing the other image.  
ZNCC is a measure of similarity between two signals or images.  
3. **Plot Results:**  
plot_metric: This function plots a given metric across slices of the image, comparing the three pairs (fixed vs. moving, fixed vs. 25% downsampled, and fixed vs. 50% downsampled).

In [ ]:
import os
import nibabel as nib
import numpy as np
from scipy.stats import entropy
from matplotlib import pyplot as plt
from matplotlib import rcParams


# Load NIfTI files
fixed_nifti_path = '/your/data/directory/fixed(template)/fixed_image.nii.gz'
fixed_nifti = nib.load(fixed_nifti_path )
dir_path = os.path.dirname(fixed_nifti_path)
moving_nifti = nib.load('/your/data/directory/moving(pre-registration)/moving_image.nii.gz')
ants_25ds_nifti = nib.load('/your/data/directory/25%-down-sample-registered(post-registration)/image.nii.gz'')
ants_50ds_nifti = nib.load('/your/data/directory/50%-down-sample-registered(post-registration)/image.nii.gz')


# Get data from NIfTI objects
fixed_data = fixed_nifti.get_fdata()
moving_data = moving_nifti.get_fdata()
ants_25ds_data = ants_25ds_nifti.get_fdata()
ants_50ds_data = ants_50ds_nifti.get_fdata()

# Calculate MI and ZNCC
def calculate_metrics(data1, data2):
    mi = []
    zncc = []

    for i in range(data1.shape[2]):
        # Get slices
        slice1 = data1[:, :, i]
        slice2 = data2[:, :, i]

        # Ignore slices that contain only zeros
        if np.count_nonzero(slice1) == 0 or np.count_nonzero(slice2) == 0:
            continue

        # MI
        joint_hist = np.histogram2d(slice1.ravel(), slice2.ravel(), bins=20)[0]
        joint_hist += 1e-5  # small constant; prevents log(0)
        joint_hist /= np.sum(joint_hist)  # normalize joint histogram to make it a joint PDF
        
        # Compute marginal histograms (i.e., the PDFs of each image)
        hist1 = np.sum(joint_hist, axis=0)
        hist2 = np.sum(joint_hist, axis=1)
        
        # Compute entropies
        h1 = entropy(hist1)
        h2 = entropy(hist2)
        joint_h = entropy(joint_hist.ravel())

        # Compute mutual information
        mi_slice = h1 + h2 - joint_h

        # Normalized Mutual Information
        normalized_mi = mi_slice / np.sqrt(h1 * h2)
        mi.append(normalized_mi)

        # ZNCC
        zncc_slice = np.sum((slice1 - np.mean(slice1)) * (slice2 - np.mean(slice2))) / (np.sqrt(np.sum((slice1 - np.mean(slice1))**2)) * np.sqrt(np.sum((slice2 - np.mean(slice2))**2)))
        zncc.append(zncc_slice)

    return mi, zncc

# Calculate metrics for the two pairs
mi_moving, zncc_moving = calculate_metrics(fixed_data, moving_data)
mi_ants_25ds, zncc_ants_25ds = calculate_metrics(fixed_data, ants_25ds_data)
mi_ants_50ds, zncc_ants_50ds = calculate_metrics(fixed_data, ants_50ds_data)

# Function to plot each metric and display min, max, median values
def plot_metric(metric_values1, metric_values2, metric_values3, metric_name):
    plt.figure(figsize=(10, 6))
    plt.plot(metric_values1, label='Moving', linewidth=2)
    plt.plot(metric_values2, label='25% downsize ANTs', linewidth=2)
    plt.plot(metric_values3, label='50% downsize ANTs', linewidth=2)
    plt.title(f'{metric_name}')
    plt.xlabel('Slice Number')
    plt.ylabel('Similarity')
    plt.legend()
    plt.savefig(os.path.join(dir_path  ,f'{metric_name}.png'), dpi=1200)
    plt.show()

# Plot each metric
plot_metric(mi_moving, mi_ants_25ds, mi_ants_50ds, 'MI')
plot_metric(zncc_moving, zncc_ants_25ds, zncc_ants_50ds, 'ZNCC')


## Remarks:
- Ensure that the paths to the NIfTI files are correct. Update the placeholders like '/your/data/directory/...' with the actual paths to your NIfTI files.
- The code uses normalized MI, which divides the MI by the geometric mean of the entropies of the two images. This normalization ensures the MI value lies between 0 (no mutual information) and 1 (perfect alignment).
- ZNCC values range between -1 and 1. A value of 1 indicates perfect alignment, 0 indicates no alignment, and -1 indicates perfect misalignment.
- It's essential to interpret the results in the context of your specific application. While higher MI and ZNCC indicate better alignment, the absolute values might vary based on the quality and characteristics of the images.
- Remember to check the saved plots in the directory specified by dir_path for a visual understanding of the alignment quality across slices.